# 서비스 진행상황 가시화
## 데이터준비

In [2]:
from NewsAnalyzer.PreProcesser import preprocesser

pp = preprocesser()

file_path = "data/2022-04-18 news data.csv"

df = pp.load_data(file_path)
df

,카테고리,언론사,기사제목,날짜,본문,기사링크
0,정치,SBS,[단독] '6억 집을 1억에'…이창용 한은 총재 후보자 다운계약 의혹,2022.04.18.,"\n\n\n\n\n이창용 한국은행 총재 후보자가 지난 2002년, 서울 중구 신당동...",https://news.naver.com//main/read.naver?mode=L...
1,정치,중앙일보,"노정희 선관위원장, '소쿠리 대선' 논란 44일만에 사퇴",2022.04.18.,\n\n\n\n\n\t\n\t 노정희 중앙선거관리위원회위원장이 18일 사의를 표명...,https://news.naver.com//main/read.naver?mode=L...
2,정치,YTN,단거리 발사에 김정은 참석...전술핵 탑재 노렸나,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]김정은 국무위원장이 단...,https://news.naver.com//main/read.naver?mode=L...
3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...
4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...
...,...,...,...,...,...,...
4641,세계,국제신문,전 세계 물가 폭등 아우성…30년만에 ‘大인플레’ 시대 진입,2022.04.18.,\n\n\n\n\nOECD 2월 상승률 7.7% 급등- 우크라發 에너지가격 큰 폭 ...,https://news.naver.com//main/read.naver?mode=L...
4642,세계,국민일보,물·전기 없이 13만명 ‘절망의 항전’… 마리우폴 ‘참극’,2022.04.18.,\n\n\n\n\n초토화 작전에 건물 90% 이상 파괴우크라군 최악 포위전 속 결사...,https://news.naver.com//main/read.naver?mode=L...
4643,세계,중앙일보,러 “마리우폴 함락” 주장…우크라 “방어군 버티고 있다”,2022.04.18.,\n\n\n\n\n 우크라이나 동남부 돈바스 지역에서 친러 분리주의 무장대원들이 ...,https://news.naver.com//main/read.naver?mode=L...
4644,세계,중앙일보,"침몰한 러의 자존심…미 “푸틴, 핵 쓸 가능성 커져”",2022.04.18.,\n\n\n\n\n\t\n\t 지난 14일 러시아 해군 흑해함대의 기함인 모스크바...,https://news.naver.com//main/read.naver?mode=L...


In [11]:
df.drop(2,inplace=True)
df

,카테고리,언론사,기사제목,날짜,본문,기사링크
3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...
4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...
5,정치,YTN,등 떠밀려 첫 사과?...'5·18 폄훼' 김진태 결국 경선,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]국민의힘 강원도지사 공...,https://news.naver.com//main/read.naver?mode=L...
6,정치,디지털타임스,국힘 강원도지사 결국 김진태-황상무 경선…金 단식투쟁-사과문 통했나,2022.04.18.,"\n\n\n\n\n국힘 공천관리위, 당초 예비후보 2인서 김진태 배제-황상무 단수추...",https://news.naver.com//main/read.naver?mode=L...
7,정치,디지털타임스,"김재원 ""사퇴 요구만 받아"" 유영하 ""제안 일방적"" 대구시장 경선 단일화 결렬",2022.04.18.,"\n\n\n\n\n金-柳 단일화 회동 후 결렬 책임공방…홍준표 견제 시도 '빈손',...",https://news.naver.com//main/read.naver?mode=L...
...,...,...,...,...,...,...
4641,세계,국제신문,전 세계 물가 폭등 아우성…30년만에 ‘大인플레’ 시대 진입,2022.04.18.,\n\n\n\n\nOECD 2월 상승률 7.7% 급등- 우크라發 에너지가격 큰 폭 ...,https://news.naver.com//main/read.naver?mode=L...
4642,세계,국민일보,물·전기 없이 13만명 ‘절망의 항전’… 마리우폴 ‘참극’,2022.04.18.,\n\n\n\n\n초토화 작전에 건물 90% 이상 파괴우크라군 최악 포위전 속 결사...,https://news.naver.com//main/read.naver?mode=L...
4643,세계,중앙일보,러 “마리우폴 함락” 주장…우크라 “방어군 버티고 있다”,2022.04.18.,\n\n\n\n\n 우크라이나 동남부 돈바스 지역에서 친러 분리주의 무장대원들이 ...,https://news.naver.com//main/read.naver?mode=L...
4644,세계,중앙일보,"침몰한 러의 자존심…미 “푸틴, 핵 쓸 가능성 커져”",2022.04.18.,\n\n\n\n\n\t\n\t 지난 14일 러시아 해군 흑해함대의 기함인 모스크바...,https://news.naver.com//main/read.naver?mode=L...


In [2]:
title = df["기사제목"]
news = df["본문"]
link = df["기사링크"]

## 키워드 뽑기

In [10]:
from krwordrank.word import summarize_with_keywords

def listToString(str_list): #리스트를 문자열로 반환 (특수문자 제거 후 추출된 명사 입력)
    result = ""
    for s in str_list:
        result += s + " "
    string = result.strip()
    return string

def kwordrank(string): #해당 함수에서 불용어 제거 
    stop_words_list=[]
    stop_words_file = open("korean_stop_words.txt","r",encoding='utf-8')
    for line in stop_words_file.readlines():
        stop_words_list.append(line.rstrip())
        stop_words_file.close()
    stopwords=stop_words_list
    texts = [string]
    keywords = summarize_with_keywords(texts, min_count=10, max_length=10,beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
    passwords = {word:score for word, score in sorted(
        keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}
    return passwords


2. 전처리 진행한 데이터 집어넣고 키워드 뽑기

In [13]:
title_n = pp.prep(file_path=file_path,title=True,tokenizing=False)

training was done. used memory 1.315 Gb
all cohesion probabilities was computed. # words = 132468
all branching entropies was computed # words = 237707
all accessor variety was computed # words = 237707


In [6]:
from NewsAnalyzer.Analyzer import analysis
an =analysis()

In [14]:
k = an.listToString(title_n)
s = an.kwordrank(k)

scan vocabs ... 
num vocabs = 2415
done = 9 Early stopped.


In [11]:
title_s = listToString(title_n)

In [12]:
key_title = kwordrank(title_s)
key_title

scan vocabs ... 
num vocabs = 2415
done = 9 Early stopped.


{'정호영': 6.6895127136670665,
 '해제': 4.9649957509194,
 '검수완박': 4.050311230065861,
 '거리두기': 3.960031571256514,
 '만에': 3.9215912092507796,
 '후보': 3.2884152935961457,
 '검찰': 3.2226923251685027,
 '국민': 3.1014433829535255,
 '시장': 3.0683206680452475,
 '김오수': 3.0554897274123825,
 '이은해': 3.0363886052025557,
 '삼성': 2.988681139926485,
 '민주': 2.974158588891276,
 '코로나': 2.96637602878558,
 '의혹': 2.8919178352972787,
 '확진': 2.783554380763745,
 '종합': 2.7728616047491035,
 '우크라': 2.6639330406236508,
 '조현수': 2.629271382040585,
 '서울': 2.407643079810572,
 '조국': 2.1083073542128834,
 '경찰': 2.053223482437182,
 '수사': 2.010010602284594,
 '일상': 1.9982481417952425,
 'LG': 1.9541889381544162,
 '19': 1.8566039323594818,
 '논란': 1.827778933894403,
 '공개': 1.8270437299371016,
 '부산': 1.8256118769363971,
 '회의': 1.8236334728765677,
 '국힘': 1.8168435945975503,
 '10': 1.752149028983383,
 '조민': 1.7153705176572318,
 '20': 1.6908904532467561,
 '살인': 1.6591611198135157,
 '정부': 1.6463983368955706,
 '한동훈': 1.6359720570022787,
 '대통':

-> 뉴스 본문이 들어가면 결과가 별로임
-> 기사 제목만을 가지고 키워드를 뽑는게 빠르고 성능이 좋음

In [32]:
keywords = key_title
keywords

{'정호영': 6.6895127136670665,
 '해제': 4.9649957509194,
 '검수완박': 4.050311230065861,
 '거리두기': 3.960031571256514,
 '만에': 3.9215912092507796,
 '후보': 3.2884152935961457,
 '검찰': 3.2226923251685027,
 '국민': 3.1014433829535255,
 '시장': 3.0683206680452475,
 '김오수': 3.0554897274123825,
 '이은해': 3.0363886052025557,
 '삼성': 2.988681139926485,
 '민주': 2.974158588891276,
 '코로나': 2.96637602878558,
 '의혹': 2.8919178352972787,
 '확진': 2.783554380763745,
 '종합': 2.7728616047491035,
 '우크라': 2.6639330406236508,
 '조현수': 2.629271382040585,
 '서울': 2.407643079810572,
 '조국': 2.1083073542128834,
 '경찰': 2.053223482437182,
 '수사': 2.010010602284594,
 '일상': 1.9982481417952425,
 'LG': 1.9541889381544162,
 '19': 1.8566039323594818,
 '논란': 1.827778933894403,
 '공개': 1.8270437299371016,
 '부산': 1.8256118769363971,
 '회의': 1.8236334728765677,
 '국힘': 1.8168435945975503,
 '10': 1.752149028983383,
 '조민': 1.7153705176572318,
 '20': 1.6908904532467561,
 '살인': 1.6591611198135157,
 '정부': 1.6463983368955706,
 '한동훈': 1.6359720570022787,
 '대통':

### 임의의 키워드 하나를 선택하여 진행.

In [10]:
keyword = "거리두기"

키워드를 선정하면 비슷한 키워드를 뽑아내는 과정

In [8]:
import pandas as pd
from konlpy.tag import Okt
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def show_relevant_keyword(keyword, data): # 키워드와 사용할 뉴스 데이터를 인자로 입력받음
    
    # 에러를 막기 위해 'english'로 설정하였음. 
    # max_features는 5000개로 설정함
    tv = TfidfVectorizer(stop_words = 'english', max_features = 5000)
    x = tv.fit_transform(data)

    # words에는 feature가 된 단어들이 5000개 담겨 있음. 
    words = tv.get_feature_names() 

    # 불용어 리스트 불러오기
    stopwords = pd.read_csv('./korean_stop_words.txt', encoding = 'utf8')['아']

    ############## SVD 특이값 분해 ################
    from sklearn.decomposition import TruncatedSVD

    # n_components 는 max_features보다 적어야 함. 
    # 임의로 300개로 설정하였음
    svd = TruncatedSVD(n_components = 300, random_state = 1234)
    word_idx = words.index(keyword) # 사용자에게 입력받은 키워드의 인덱스 위치 확인
    svd.fit(x)

    kw_idx = svd.components_[:, word_idx].argmax()

    relevant_words_df = pd.DataFrame({'단어': words, 'loading': svd.components_[kw_idx]})
    
    ########### 상위 10가지 키워드 보여주기 ##########
    
    # 불용어 처리
    rel_words_df = relevant_words_df[relevant_words_df['단어'] != keyword]
    drop_index = []
    for i in rel_words_df['단어']:
        if i in list(stopwords):
            idx = rel_words_df[rel_words_df['단어'] == i].index[0]
            drop_index.append(idx)
    rel_words_df.drop(drop_index, inplace = True)
    
    # 상위 10개의 단어 추출
    rel_words_df = rel_words_df.sort_values('loading').tail(10)
    rel_words_df = rel_words_df.sort_values('loading', ascending = False)


    # 사용자가 입력한 키워드와 관련있는 상위 10개 단어를 데이터 프레임으로 반환
    return rel_words_df


In [11]:
# 전처리한 데이터 사용
rkw = show_relevant_keyword(keyword,title_n)

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
rkw

,단어,loading
4816,해제,0.440051
1453,만에,0.235141
4309,코로나,0.161723
499,검수완박,0.154114
869,김오수,0.145926
504,검찰,0.124013
3576,전면,0.116556
579,고검장,0.115954
3699,정호영,0.110045
4959,회의,0.108344


In [14]:
show_relevant_keyword("정호영",title_n)

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,단어,loading
3197,의혹,0.208852
733,국민의힘,0.189888
4131,청문회,0.156239
4971,후보,0.133101
3743,조국,0.129833
1126,달라,0.127383
1044,논란,0.127310
4782,합당,0.107681
1150,당선인,0.100596
4438,판단,0.099829


In [15]:
title_n

['단독 6억 집을 1억에 이창용 한은 총재 후보 자 다운 계약 의혹',
 '노정희 선관 위원장 소쿠리 대선 논란 44일 만에 사퇴',
 '단 거리 발사에 김정은 참석 전술핵 탑재 노렸나',
 '장제원 조작 위조 정호영 조국과 뭐가 같나 얘기 해보라',
 '국민의힘 국민의 당 합당 공식 선언 공동 정부 초석 종합 2보',
 '등 떠 밀려 첫 사과 5 18 폄훼 김진태 결국 경선',
 '국힘 강원도지사 결국 김진태 황상무 경선 金 단식투쟁 사과문 통했나',
 '김재원 사퇴 요구 만 받아 유영하 제안 일방적 대구 시장 경선 단일화 결렬',
 '尹 정호영 감싸자 진중권 누구 처럼 마음 의 빚 졌다 하겠지',
 '김어준 尹 조국 두고 공정 언급 말라 대권 잡았 으니 만족을',
 '제2의 현송월 김정은 곁 등장한 뉴 페이스 여성 눈길 영상',
 '안철수 협치 위해 살얼음판 걸어 인수위 존재 감은 미약',
 '김은혜 공정 상식 세울것 vs 유승민 중도 확장성 2차 토론 격돌',
 '평가 갈리는 안철수 흡수 합당 약속 지킨 것 vs 다당제 멀어 져 이슈',
 '대구 시장 여론조사 홍준표 20 30 젊은 층서 압도적 우위 고령층 은 경합',
 '문경 시장 여론조사 신현국 37 7 서원 26 1 채홍호 21 4',
 '대선 사전투표 부실 관리 노정희 중앙선 관위장 사퇴',
 '노정희 선관 위원장 사퇴 사전투표 관리 책임 통감',
 '정호영 아빠찬스 에 국힘 의견 분분 부정 팩트 無 vs 자진 사퇴',
 '국힘서도 하태경 정호영 자진사퇴 해야 불공정한 것',
 '신동욱 앵커 의 시선 어디선가 본 듯한',
 '한동훈 상속 토지 농지법 위반 지적에 선친 밭 일구며 생활',
 '민주당 경선서 통신요금 청구 지 바꿔치기 로 여론조사 왜곡',
 '김진태 부활 하나 국민의힘 강원도지사 후보 경선 선회',
 '與 충북 노영민 공천 확정 野 김재원 유영 하는 단일화 추진',
 '김현숙 남편 5년 전 없던 비상장 주식 9억대 보유',
 '한동훈 내정자 일가 상속 증여 토지 농지법 위반 의혹',

현재까지 상황
1. 키워드 뽑음 -> 워드클라우드 그릴 수 있음
2. 키워드에서 관련키워드 뽑음 -> 얘도 워드클라우드 그릴수 있음
3. 본문요약은 BART를 사용하는데 본문 하나 넣으면 요약이 아주 잘됩니다.

문제
1. 키워드와 연관된 본문 뽑아오기 -> 기사제목에서 키워드 들어간거 가져오기. (진짜 단순하게 한다면)
2. 본문에서 단어 추출하고, 키워드 연관단어와 유사도 측정해서 기사 선정 -> 미친짓 why? 시간이 너무 오래걸림
3. 전체 기사를 전부 요약하는것도 시간이 너무 오래걸림.



In [38]:
from keybert import KeyBERT

kw = KeyBERT()

In [42]:
keywords = kw.extract_keywords(news_n[1], keyphrase_ngram_range=(1, 1), stop_words=None)
keywords

[('공정과', 0.4153),
 ('더불어민주당', 0.3997),
 ('전전긍긍', 0.3929),
 ('부정의', 0.386),
 ('당신의', 0.3584)]

In [16]:
from NewsAnalyzer.Analyzer import analysis

an = analysis()

문서 요약은 전처리하지 않은 데이터를 넣는게 좋다

In [23]:
news[1]

'\n\n\n\n\n\t\n\t  노정희 중앙선거관리위원회위원장이 18일 사의를 표명했다.      노 위원장은 이날 정부 과천청사에서 비공개로 진행된 선관위 전체회의에서 “대통령 선거에서 코로나 확진자들의 사전투표 관리에 대한 책임을 통감한다”며 위원장직에서 물러날 뜻을 밝혔다.        지난 3월 8일 과천 중앙선거관리위원회에서 20대 대통령 선거 본 투표일을 하루 앞두고 담화문을 발표하며 코로나19 확진자 사전투표 혼란과 관련해 사과 인사하는 노정희 선관위원장. 연합뉴스            노 위원장은 사의를 표명하며 “국민께 심려를 끼쳐드린 것에 대해 다시 한 번 깊이 사과드리며, 지방선거가 흠 없이 치러지도록 국민 모두가 협조해 주실 것을 부탁드린다”고 말했다고 선관위가 전했다.      노 위원장의 사퇴는 지난달 5일 3ㆍ9대선의 사전투표 관리 부실로 논란을 빚은지 44일만이다.       선관위는 코로나 확진ㆍ격리자들의 대선 사전투표를 부실하게 진행하면서 논란을 자초했다. 사전투표는 확진자가 투표용지를 투표함에 직접 넣지 못하고 투표사무원에 전달하는 방식으로 진행됐는데, 그 과정에서 쇼핑백과 바구니, 택배박스 등에 담긴 투표용지가 노출되면서 ‘소쿠리 대선’ 논란을 빚었다. 일부 투표소에선 기표된 투표용지를 받은 유권자도 있었다.       특히 노 위원장이 사전투표 당일 출근하지 않은 사실까지 드러나면서 사퇴 여론이 거세졌다.       국민의힘의 대선 부실 관리의 책임을 물어 노 위원장의 사퇴를 요구해왔지만, 노 위원장은 지난달 8일 대국민담화에서 “미흡한 준비로 혼란과 불편을 끼친 점에 대해 책임을 통감한다”면서도 거취에 대해선 침묵했다. 지난달 17일엔 선관위 직원들에게 보낸 이메일에서도 “책임을 회피하지 않겠다”면서도 “목전에 다가온 지방선거를 더 이상 흔들림 없이 준비ㆍ관리하기 위해서는 위원장으로서 신중할 수밖에 없다”고 밝혔다.      한편 선관위는 이날 아들의 취업 특혜 의혹을 받은 김세환 전 사무총장에 대한 내부 특별감찰 결과 

In [24]:
an.summarizer(news[1])

'노정희 중앙선거관리위원회위원장이 18일 정부 과천청사에서 비공개로 진행된 선관위 전체회의에서 “대통령 선거에서 코로나 확진자들의 사전투표 관리에 대한 책임을 통감한다”며 위원장직에서 물러날 뜻을 밝혔다.'

In [33]:
key_title

{'정호영': 6.6895127136670665,
 '해제': 4.9649957509194,
 '검수완박': 4.050311230065861,
 '거리두기': 3.960031571256514,
 '만에': 3.9215912092507796,
 '후보': 3.2884152935961457,
 '검찰': 3.2226923251685027,
 '국민': 3.1014433829535255,
 '시장': 3.0683206680452475,
 '김오수': 3.0554897274123825,
 '이은해': 3.0363886052025557,
 '삼성': 2.988681139926485,
 '민주': 2.974158588891276,
 '코로나': 2.96637602878558,
 '의혹': 2.8919178352972787,
 '확진': 2.783554380763745,
 '종합': 2.7728616047491035,
 '우크라': 2.6639330406236508,
 '조현수': 2.629271382040585,
 '서울': 2.407643079810572,
 '조국': 2.1083073542128834,
 '경찰': 2.053223482437182,
 '수사': 2.010010602284594,
 '일상': 1.9982481417952425,
 'LG': 1.9541889381544162,
 '19': 1.8566039323594818,
 '논란': 1.827778933894403,
 '공개': 1.8270437299371016,
 '부산': 1.8256118769363971,
 '회의': 1.8236334728765677,
 '국힘': 1.8168435945975503,
 '10': 1.752149028983383,
 '조민': 1.7153705176572318,
 '20': 1.6908904532467561,
 '살인': 1.6591611198135157,
 '정부': 1.6463983368955706,
 '한동훈': 1.6359720570022787,
 '대통':

In [60]:
lst = []

for word, num in key_title.items():
    k = [word, num]
    lst.append(k)

In [61]:
lst = lst[:10]

In [62]:
lst

[['정호영', 6.6895127136670665],
 ['해제', 4.9649957509194],
 ['검수완박', 4.050311230065861],
 ['거리두기', 3.960031571256514],
 ['만에', 3.9215912092507796],
 ['후보', 3.2884152935961457],
 ['검찰', 3.2226923251685027],
 ['국민', 3.1014433829535255],
 ['시장', 3.0683206680452475],
 ['김오수', 3.0554897274123825]]

In [63]:
for word in lst:
    rkw = show_relevant_keyword(word[0],title_n)
    word.append(rkw.단어.values.tolist())

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [65]:
df = pd.DataFrame(lst)

In [68]:
df.to_csv("keyword top 10.csv")